In [1]:
!pip install pyLDAvis --quiet
!pip install chart_studio --quiet


     |████████████████████████████████| 1.6MB 13.1MB/s 
     |████████████████████████████████| 71kB 5.1MB/s 


In [2]:
import pandas as pd
import numpy as np
import time
import re
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import LatentDirichletAllocation
import gensim
from spacy.tokenizer import Tokenizer
import gensim.corpora as corpora
from gensim.models.ldamulticore import LdaMulticore
from pprint import pprint
from gensim.models.coherencemodel import CoherenceModel
import plotly.express as px
import pyLDAvis.gensim
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls
from operator import itemgetter
from ipywidgets import interact
import tqdm
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets


# supress warnings
warnings.filterwarnings("ignore")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


We will load in our preprocessed tweets from our [data cleaning notebook](https://github.com/tarrantcarter/Final_Capstone/blob/main/Modern_Motivation_Data_Cleaning_Feature_Engineering.ipynb). The csv can be found [here](https://drive.google.com/file/d/1-d-61YuocweY0F3rSnf-idL1ixbuKq4H/view?usp=sharing). 

In [4]:
# load in cleaned tweets from data cleaning notebook
tweets_cleaned = pd.read_json("/content/drive/MyDrive/Data/NLP_Capstone/motivational_tweets_cleaned.json")

In [5]:
tweets_cleaned.head()

,date,user_name,content,content_preprocessed,unigram_tokens,ngrams,ngram_tokens,nouns_only,nouns_verbs,bigrams_trigrams,bigrams_trigrams_strings
44,2021-01-17 22:13:17,LewisHowes,Know this. Everything is happening for a reaso...,know happen reason favor betterment future pai...,"[know, happen, reason, favor, betterment, futu...",know_happen_reason favor betterment future pai...,"[know_happen_reason, favor, betterment, future...","[reason, favor, betterment, future, pain, feel...","[know, reason, favor, betterment, future, pain...",[know_happen_reason],know_happen_reason
61,2021-01-15 15:28:06,LewisHowes,Protect your inner peace at all costs. Create ...,protect inner peace cost create daily practice...,"[protect, inner, peace, cost, create, daily, p...",protect inner_peace cost create daily_practice...,"[protect, inner_peace, cost, create, daily_pra...","[peace, cost, practice, communicate, stress, d...","[peace, cost, practice, communicate, stress, d...","[inner_peace, daily_practice]",inner_peace daily_practice
161,2021-01-07 16:00:29,LewisHowes,Always remember to ask for exactly what you wa...,remember ask exactly want ask love good health...,"[remember, ask, exactly, want, ask, love, good...",remember ask exactly want ask love good health...,"[remember, ask, exactly, want, ask, love, good...","[health, abundance, peace, ask, wisdom, creati...","[remember, health, abundance, peace, ask, wisd...","[health_abundance, ask_wisdom]",health_abundance ask_wisdom
274,2021-01-01 02:44:46,LewisHowes,This will be your greatest year ever. All your...,great year work start pay earn happy love deep...,"[great, year, work, start, pay, earn, happy, l...",great year work start pay earn happy love deep...,"[great, year, work, start, pay, earn, happy, l...","[year, work, start, earn, love, embrace, fear,...","[year, work, start, pay, earn, love, embrace, ...",[massive_action],massive_action
317,2020-12-30 01:38:35,LewisHowes,Be grateful for the breakdown this year. It's ...,grateful breakdown year set massive breakthrou...,"[grateful, breakdown, year, set, massive, brea...",grateful breakdown year set massive breakthrou...,"[grateful, breakdown, year, set, massive, brea...","[year, breakthrough, money, mission, get, rela...","[year, set, breakthrough, money, come, mission...",[real_friend],real_friend


In [6]:
tweets_cleaned.applymap(type)

,date,user_name,content,content_preprocessed,unigram_tokens,ngrams,ngram_tokens,nouns_only,nouns_verbs,bigrams_trigrams,bigrams_trigrams_strings
44,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
61,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
161,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
274,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
317,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
...,...,...,...,...,...,...,...,...,...,...,...
977598,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
977599,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
977606,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
977608,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>


In [7]:
tweets_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 621449 entries, 44 to 977610
Data columns (total 11 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   date                      621449 non-null  datetime64[ns]
 1   user_name                 621449 non-null  object        
 2   content                   621449 non-null  object        
 3   content_preprocessed      621449 non-null  object        
 4   unigram_tokens            621449 non-null  object        
 5   ngrams                    621449 non-null  object        
 6   ngram_tokens              621449 non-null  object        
 7   nouns_only                621449 non-null  object        
 8   nouns_verbs               621449 non-null  object        
 9   bigrams_trigrams          621449 non-null  object        
 10  bigrams_trigrams_strings  621449 non-null  object        
dtypes: datetime64[ns](1), object(10)
memory usage: 56.9+ MB


In [8]:
tweets_cleaned.shape

(621449, 11)

# Topic Modeling

In [13]:
# create dictionary
id2word = corpora.Dictionary(tweets_cleaned['unigram_tokens'])
# create texts corpus
texts = tweets_cleaned['unigram_tokens']
# term document frequency
corpus = [id2word.doc2bow(text) for text in texts]
# print first 30 tuples from corpus
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1)]


In [14]:
# number of topics
num_topics = 10
# build LDA model
base_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# # print the keyword in the 10 topics
# pprint(base_model.print_topics())
# doc_lda = base_model[corpus]

In [15]:
# filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]

# create topic sorted by 10 most relevent words
topics = [' '.join(t[0:10]) for t in words]


# print most relevent words for each topic
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
look mean peale energy strength real open henry life vincent

------ Topic 1 ------
love want life lincoln ask abraham give live let u

------ Topic 2 ------
dyer thomas mind god edison feeling follow bind roosevelt eye

------ Topic 3 ------
day success time life learn hold new failure fear robert

------ Topic 4 ------
good man come world thing life great know act treat

------ Topic 5 ------
power heart unknown self lead life control u look trust

------ Topic 6 ------
change people like light time thing life love know great

------ Topic 7 ------
way choice love happen thing believe make fall attract place

------ Topic 8 ------
time get john better today succeed think win dream proverb

------ Topic 9 ------
dyer amp think u thought einstein albert peace happiness life



In [16]:
# Compute Perplexity
## a measure of how good the model is. lower the better
base_perplexity = base_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=base_model, texts=tweets_cleaned['unigram_tokens'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)


Perplexity:  -7.194890878960951

Coherence Score:  0.42821470373489356


In [17]:
# topic distance visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(base_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.137374  0.057895       1        1  12.401606
1     -0.105605  0.066358       2        1  11.661596
6     -0.137261 -0.096341       3        1  11.484403
3     -0.044100 -0.144930       4        1  10.845585
9      0.299946  0.060289       5        1  10.441628
8     -0.007670 -0.341081       6        1   9.275158
7     -0.145858  0.132404       7        1   9.064219
5      0.027814  0.145506       8        1   8.893433
0     -0.049370  0.131852       9        1   8.267247
2      0.299479 -0.011952      10        1   7.665126, topic_info=         Term           Freq          Total Category  logprob  loglift
2060     dyer  106959.000000  106959.000000  Default  30.0000  30.0000
46       good   31570.000000   31570.000000  Default  29.0000  29.0000
180       way   29898.000000   29898.000000  Default  28.0000  28.0000
94     change   24402.000000   24402.000000  Default  27.0000  27.0000
358       man   37373.000000   37373.000000  Default  26.0000  26.0000
...       ...            ...            ...      ...      ...      ...
167     judge    3233.604364    7141.689215  Topic10  -4.6122   1.7761
17     reason    3002.606866    6300.261717  Topic10  -4.6864   1.8274
124      life    4160.463461   62758.334512  Topic10  -4.3602  -0.1452
328   thought    3024.004745   13367.326824  Topic10  -4.6793   1.0823
50       love    2527.561841   38133.557901  Topic10  -4.8586  -0.1454

[653 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
267       1  0.002627  ability
267       2  0.000606  ability
267       4  0.984456  ability
267       5  0.001415  ability
267       7  0.001617  ability
...     ...       ...      ...
4851      5  0.999544     yogi
4851      6  0.000347     yogi
905       8  0.998802    young
4863      3  0.014862      zen
4863     10  0.984629      zen

[1679 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 7, 4, 10, 9, 8, 6, 1, 3])

In [18]:
get_document_topics = [base_model.get_document_topics(item) for item in corpus]

In [19]:
len(get_document_topics)

621449

In [20]:
get_document_topics[:20]

[[(0, 0.2733185), (3, 0.22056545), (7, 0.28862822), (8, 0.18747967)],
 [(0, 0.16377401),
  (1, 0.12425981),
  (3, 0.056439172),
  (7, 0.098901086),
  (8, 0.2993697),
  (9, 0.23985928)],
 [(0, 0.14730129),
  (1, 0.31619433),
  (2, 0.09452664),
  (4, 0.107680075),
  (5, 0.048008587),
  (7, 0.2041978),
  (9, 0.06904622)],
 [(0, 0.12901646),
  (1, 0.17792825),
  (2, 0.049956914),
  (3, 0.21398056),
  (4, 0.06766333),
  (5, 0.12272997),
  (7, 0.17966402),
  (9, 0.05036387)],
 [(0, 0.106191345),
  (1, 0.06657115),
  (2, 0.094773695),
  (3, 0.10476098),
  (4, 0.14881118),
  (5, 0.1872235),
  (6, 0.06365736),
  (7, 0.014526975),
  (8, 0.14462705),
  (9, 0.06885677)],
 [(0, 0.04316089),
  (1, 0.2603997),
  (2, 0.05111535),
  (3, 0.06625228),
  (4, 0.1973748),
  (6, 0.108248316),
  (7, 0.06675206),
  (9, 0.19836073)],
 [(1, 0.3188603),
  (2, 0.12955335),
  (3, 0.078008),
  (4, 0.1333913),
  (5, 0.08578342),
  (7, 0.18304686),
  (9, 0.057067934)],
 [(0, 0.014285888),
  (1, 0.01428651),
  (2, 0.15

In [21]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           alpha=a,
                                           eta=b,
                                           random_state=222)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [22]:
# start preprocess runtime
start_time = time.time() 

# Topics range
min_topics = 3
max_topics = 10
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = [.01,.1,.5,1,'symmetric','asymmetric']
# Beta parameter
beta = [.01,.1,.5,1,'symmetric']

# model results dict
model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=210)

    # iterate through number of topics
    for k in topics_range:
        # iterate through alpha values
        for a in alpha:
            # iterare through beta values
            for b in beta:
                # get the coherence score for the given parameters
                cv = compute_coherence_values(corpus=corpus, dictionary=id2word, 
                                              k=k, a=a, b=b)
                # Save the model results
                model_results['Topics'].append(k)
                model_results['Alpha'].append(a)
                model_results['Beta'].append(b)
                model_results['Coherence'].append(cv)
                
                pbar.update(1)
    lda_tuning_results = pd.DataFrame(model_results)                
    lda_tuning_results.to_csv('/content/drive/MyDrive/Data/NLP_Capstone/lda_tuning_no_ngrams_3-10.csv', index=False)
    pbar.close()

# print preprocess runtime
print(time.strftime(f'%H hours, %M minutes, %S seconds', time.gmtime(time.time() - start_time)))

100%|██████████| 210/210 [5:24:41<00:00, 92.77s/it]

05 hours, 24 minutes, 41 seconds


In [23]:
lda_tuning_results.sort_values('Coherence',ascending=False).head(30)

,Topics,Alpha,Beta,Coherence
190,9,0.5,0.01,0.461507
199,9,1,symmetric,0.460231
194,9,0.5,symmetric,0.451668
191,9,0.5,0.1,0.450867
196,9,1,0.1,0.450447
162,8,0.5,0.5,0.449808
192,9,0.5,0.5,0.449331
165,8,1,0.01,0.449074
185,9,0.1,0.01,0.447365
193,9,0.5,1,0.443921


In [24]:
# number optimal of topics
num_topics = 8
# build optimal LDA model
optimal_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                        alpha=.5,
                                        eta=.1,
                                        random_state=222
                                        )
# filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in optimal_model.print_topics()]

# create topic sorted by 10 most relevent words
topics = [' '.join(t[0:10]) for t in words]


# print most relevent words for each topic
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
time man choice right work proverb strong lose moment love

------ Topic 1 ------
love mind peace give dream strength happen place lie happy

------ Topic 2 ------
life amp way day person learn live find well matter

------ Topic 3 ------
good people want try self result action jim attract far

------ Topic 4 ------
like look change fear small believe roosevelt fail henry important

------ Topic 5 ------
dyer u think thing act thought feel thomas man great

------ Topic 6 ------
world success man einstein albert choose create opportunity churchill franklin

------ Topic 7 ------
know come heart light get let lincoln thing abraham lead



In [25]:
# Compute Perplexity
## a measure of how good the model is. lower the better
base_perplexity = optimal_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=optimal_model, texts=tweets_cleaned['unigram_tokens'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)


Perplexity:  -7.1426995207016395

Coherence Score:  0.4421384563692554


In [26]:
# topic distance visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(optimal_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.241999 -0.132668       1        1  13.266446
7     -0.196568  0.122225       2        1  13.139130
5     -0.092043  0.008941       3        1  12.912703
0     -0.114912 -0.151828       4        1  12.504983
4     -0.120315  0.182134       5        1  12.480053
1     -0.081330 -0.182527       6        1  12.315146
3      0.231026  0.265587       7        1  11.863117
6      0.132143 -0.111865       8        1  11.518422, topic_info=             Term           Freq          Total Category  logprob  loglift
2060         dyer  113678.000000  113678.000000  Default  30.0000  30.0000
124          life   62931.000000   62931.000000  Default  29.0000  29.0000
188          time   37214.000000   37214.000000  Default  28.0000  28.0000
46           good   33900.000000   33900.000000  Default  27.0000  27.0000
93            amp   42168.000000   42168.000000  Default  26.0000  26.0000
...           ...            ...            ...      ...      ...      ...
679       problem    5943.919372    8172.133940   Topic8  -4.4107   1.8429
358           man   10875.447722   31828.655234   Topic8  -3.8066   1.0874
954   achievement    3816.525363    4634.522649   Topic8  -4.8538   1.9670
346       failure    4541.336860    7295.961262   Topic8  -4.6799   1.6871
510         power    4869.740632   14177.331175   Topic8  -4.6101   1.0926

[570 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
468       4  0.018541    able
468       5  0.012714    able
468       6  0.001324    able
468       7  0.966248    able
468       8  0.000795    able
...     ...       ...     ...
4851      2  0.997826    yogi
4851      3  0.002102    yogi
4863      7  0.999790     zen
240       2  0.999513     zig
241       2  0.999447  ziglar

[1890 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 8, 6, 1, 5, 2, 4, 7])

In [27]:
# number optimal of topics
num_topics = 7
# build optimal LDA model
optimal_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                        alpha=.5,
                                        eta=.1,
                                        random_state=222
                                        )
# filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in optimal_model.print_topics()]

# create topic sorted by 10 most relevent words
topics = [' '.join(t[0:10]) for t in words]


# print most relevent words for each topic
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
time get man love choice right work proverb make moment

------ Topic 1 ------
heart love let mind peace give thomas strength dream place

------ Topic 2 ------
life amp way day look learn person find change new

------ Topic 3 ------
good people want try self john result action attract far

------ Topic 4 ------
like fear end small great believe fall succeed fail roosevelt

------ Topic 5 ------
dyer u thing think thought act great man lincoln abraham

------ Topic 6 ------
know come world success man create choose albert einstein unknown



In [28]:
# Compute Perplexity
## a measure of how good the model is. lower the better
base_perplexity = optimal_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=optimal_model, texts=tweets_cleaned['unigram_tokens'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)


Perplexity:  -7.157830286551267

Coherence Score:  0.3994161708893615


In [29]:
# topic distance visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(optimal_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.041940 -0.174656       1        1  15.085353
0     -0.156663  0.146636       2        1  14.919818
5      0.034675  0.282713       3        1  14.548581
1     -0.223151  0.033887       4        1  14.309588
3      0.191891 -0.079369       5        1  14.159444
6     -0.084959 -0.238961       6        1  13.505422
4      0.280147  0.029751       7        1  13.471795, topic_info=        Term           Freq          Total Category  logprob  loglift
2060    dyer  115027.000000  115027.000000  Default  30.0000  30.0000
124     life   63086.000000   63086.000000  Default  29.0000  29.0000
188     time   36998.000000   36998.000000  Default  28.0000  28.0000
46      good   32649.000000   32649.000000  Default  27.0000  27.0000
224     like   26589.000000   26589.000000  Default  26.0000  26.0000
...      ...            ...            ...      ...      ...      ...
440   inside    4662.567870    6091.077384   Topic7  -4.8102   1.7373
64     great    8108.575070   22406.325975   Topic7  -4.2568   0.9882
160     step    5487.027649    9998.017238   Topic7  -4.6474   1.4046
94    change    6597.602515   23946.711747   Topic7  -4.4630   0.7154
162    think    5055.984842   29031.347576   Topic7  -4.7292   0.2568

[526 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
267       1  0.145564  ability
267       2  0.791157  ability
267       3  0.000185  ability
267       4  0.007778  ability
267       5  0.001852  ability
...     ...       ...      ...
4851      2  0.000230     yogi
4851      3  0.025016     yogi
4851      6  0.974689     yogi
4863      5  0.999832      zen
241       7  0.999272   ziglar

[1623 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 6, 2, 4, 7, 5])